In [1]:
from tkinter import *
from PIL import ImageTk, Image,ImageFilter
from tkinter import filedialog
import os
from tkinter.filedialog import askdirectory
import numpy
from imshowpair import *
import matplotlib.pyplot as plt
from skimage.util import random_noise
from math import log10, sqrt
import math
from tkinter import messagebox
import scipy
from scipy.ndimage import gaussian_filter

In [2]:
window =Tk()
window.title("Image Converter")
window.geometry("1200x660")
window.resizable(width=False,height=False)
window.config(bg="#E8E8E8")

def open_file ():
    global file
    file = filedialog.askopenfilename()
    e1['state']='normal' ; e1.delete(0,END) ; e1.insert(0,file)
    e2['state']='normal'; e2.delete(0,END) ;e3['state']='normal'; e3.delete(0,END) 
    e4['state']='normal'; e4.delete(0,END) ;e5['state']='normal'; e5.delete(0,END) 
    e6['state']='normal'; e6.delete(0,END) ;e7['state']='normal'; e7.delete(0,END) 
    e8['state']='normal'; e8.delete(0,END) ;e9['state']='normal'; e9.delete(0,END) 
    e11['state']='normal'; e11.delete(0,END) ;btn_get['state'] = 'normal'
    btn_crop['state'] = 'normal';btn_resize['state'] = 'normal'
    btn_getcl.config(state='normal');btn_rotate['state'] = 'normal'
    btn_merge['state'] = 'normal';drop.config(state='normal');btn_change['state'] = 'normal'
    e12['state']='normal'; e12.delete(0,END) ;btn_bruit['state'] = 'normal';btn_filtre['state'] = 'normal'
    drop3.config(state='normal');drop4.config(state='normal');drop2.config(state='normal')
    image=Image.open(file)
    if image.mode=="L" or image.mode=="1":
        btn_dyn['state'] = 'normal';btn_ega['state'] = 'normal'
    btn_qual['state'] = 'normal';e13['state']='normal'; e13.delete(0,END)
    e14['state']='normal'; e14.delete(0,END);e15['state']='normal'; e15.delete(0,END)

def ouvre():
    image=Image.open(filedialog.askopenfilename())
    return image

def image_param(image):
    return image.format,image.size,image.mode,image.palette

def save_image(img,image,path,op):
    filename=os.path.splitext(path)[0]+op+f".{image_param(image)[0]}"
    img.save(filename)

def redimenssion(path):
    image=Image.open(path)
    x=int(e2.get())
    y=int(e3.get())
    image.thumbnail((x,y))
    image.show()
    resp = messagebox.askquestion("Save","Do You Want To Save This Work ?")
    if (resp == "yes"):
        img=image
        save_image(img,image,path,"_thumbnail")
        
def color_sys(path):
    image=Image.open(path)
    color =image.mode
    e4.insert(0,color)

def color_system(path):
    color=["RGB","L","1","CMYK"]
    image=Image.open(path)
    org=image_param(image)[2]
    v=click.get()
    if v==org:
        resp = messagebox.showinfo("Error","You Chosed the same System color !")
    else :
        img=image.convert(v)
        img.show()
        resp = messagebox.askquestion("Save","Do You Want To Save This Work ?")
        if (resp == "yes"):
            save_image(img,image,path,f"_Color_{v}")
    
def geometry(path):
    image=Image.open(path)
    retation=int(e5.get())
    img=image.rotate(retation)
    img.show()
    resp = messagebox.askquestion("Save","Do You Want To Save This Work ?")
    if (resp == "yes"):
        save_image(img,image,path,"_rotation")
        
def decoper(path):
    image=Image.open(path)
    (x1,y1,x2,y2)=(int(e6.get()),int(e7.get()),int(e8.get()),int(e9.get()))
    cord=(x1,y1,x2,y2)
    img=image.crop(cord)
    img.show()
    resp = messagebox.askquestion("Save","Do You Want To Save This Work ?")
    if (resp == "yes"):
        save_image(img,image,path,"_crop")

def merge(path):
    A=Image.open(path)
    B=ouvre()
    B=B.resize(A.size)
    T=e11.get()
    if T=="":
        resp = messagebox.showinfo("Error","Value Not Valide !")
    else:
        T=float(T)
        if T<=0 or T>=1 :
            resp = messagebox.showinfo("Error","Value Not Valide !")
        else:
            C=Image.blend(A,B,T)
            C.show()
            resp = messagebox.askquestion("Save","Do You Want To Save This Work ?")
            if (resp == "yes"):
                save_image(C,A,path,"_merge")

def bruit(path):
    image=Image.open(path)
    image_arr=numpy.array(image)
    typ=click2.get()
    noise_img = random_noise(image_arr, mode=typ)
    noise_img = np.array(255*noise_img, dtype = 'uint8')
    noise_img= Image.fromarray(noise_img)
    noise_img.show()
    resp = messagebox.askquestion("Save","Do You Want To Save This Work ?")
    if (resp == "yes"):
        save_image(noise_img,image,path,f"_bruit_{typ}")  
        
def filtrage(path):
    image=Image.open(path)
    image_arr=numpy.array(image)
    mod=click3.get()
    dim=int(click4.get())
    color=image.mode
    if mod=="Moyenneur":
        img=image.filter(ImageFilter.BoxBlur(dim))
        img.show()
        resp = messagebox.askquestion("Save","Do You Want To Save This Work ?")
        if (resp == "yes"):
            save_image(img,image,path,"_filtreMed") 
    elif mod=="Median":
        img=image.filter(ImageFilter.MedianFilter(dim))
        img.show()
        resp = messagebox.askquestion("Save","Do You Want To Save This Work ?")
        if (resp == "yes"):
            save_image(img,image,path,"_filtreMoy") 
    elif mod=="Gaussian":
        s=float(e12.get())
        arr=scipy.ndimage.filters.gaussian_filter(image_arr, sigma=s)
        img=Image.fromarray(arr)
        img.show()
        resp = messagebox.askquestion("Save","Do You Want To Save This Work ?")
        if (resp == "yes"):
            save_image(img,image,path,"_filtreGaus")
            
def pnsr(orig,fltr):
    mse = numpy.mean((orig.astype(numpy.int8) - fltr.astype(numpy.int8)) ** 2)
    if(mse == 0):
        return 100
    max_pixel = 255.0
    psnr = 10 * log10(max_pixel / sqrt(mse))
    return psnr

def histo(arr):
    list_arr=arr.ravel().tolist()
    list_deg=[];list_val=[]
    for i in range(256):
        list_deg.append(i)
        list_val.append(list_arr.count(i))
    return list_deg,list_val

def hito_rgb(arr):
    r,g,b=separ()
    r=histo(r)[1]
    g=histo(g)[1]
    b=histo(b)[1]
    return r,g,b

def histo_cum(arr):
    h=[]
    hi=histo(arr)[1]
    for i in range(len(hi)):
        h.append((1/(arr.shape[0]*arr.shape[1]))*(sum(hi[0:i+1])))
    return h

def histo_cum_rgp(path):
    image=Image.open(path)
    r,g,b=image.split()
    r=histo_cum(r)
    g=histo_cum(g)
    b=histo_cum(b)
    return r,g,b


def dynamic(path):
    image=Image.open(path)
    image_arr=numpy.array(image)
    Inew=image_arr.copy()
    Inew=(image_arr-image_arr.min())*(255/(image_arr.max()-image_arr.min()))
    Inew=Image.fromarray(Inew)
    Inew.show()
    resp = messagebox.askquestion("Save","Do You Want To Save This Work ?")
    if (resp == "yes"):
        save_image(Inew,image,path,"_normalise")

def egalisation(path):
    image=Image.open(path)
    image_arr=numpy.array(image)
    Inew=image_arr.copy()
    liste=histo_cum(image_arr)
    for i in range(image_arr.shape[1]):
        for j in range(image_arr.shape[0]):
            Inew[j][i]=liste[image_arr[j][i]]*((2**8)-1)
    Inew=Image.fromarray(Inew)
    Inew.show()
    resp = messagebox.askquestion("Save","Do You Want To Save This Work ?")
    if (resp == "yes"):
        save_image(Inew,image,path,"_egalise")   

def qualite(orig,fltr):
    list_arr=orig.ravel().tolist()
    v=(1/(orig.shape[0]*orig.shape[1])*(sum(list_arr)**2))
    mse = numpy.mean((orig.astype(numpy.int8) - fltr.astype(numpy.int8)) ** 2)
    snr=10*math.log10(v/mse)
    print("mse = ",mse,'\n',"snr = ",snr,'\n',"pnsr = ",pnsr(orig,fltr))
                
frame1=LabelFrame(window,bg="#E8E8E8",text="Original  :",font=("",12))
frame1.grid(row=0,column=0,padx=2,pady=20,ipadx=0,ipady=10)

frame2=LabelFrame(window,bg="#E8E8E8",text="Filtred  :",font=("",12))
frame2.grid(row=0,column=1,padx=0,pady=20,ipadx=8,ipady=140)

#------------------------Original----------------------------------------------------------------

btn_get=Button(frame1,text='Upload Audio',command=open_file)
e1=Entry(frame1,width=70,state='disable')
mylabel1 = Label(frame1,text="Width :",)
e2=Entry(frame1,width=10,state='disable')
mylabel2 = Label(frame1,text="Height :",)
e3=Entry(frame1,width=10,state='disable')
btn_resize=Button(frame1,text='Resize',state='disable',command=lambda :redimenssion(file))
mylabel3 = Label(frame1,text="System Color :",)
e4=Entry(frame1,width=10,state='disable')
btn_getcl=Button(frame1,text='Get System Color',state='disable',command=lambda :color_sys(file))
mylabel4 = Label(frame1,text="Chose System Color :",)
color = ["RGB","L","1","CMYK"]
click = StringVar(window)
click.set("Color")
drop = OptionMenu(frame1, click, *color)
drop.config(state='disable')
btn_change=Button(frame1,text='Change',state='disable',command=lambda :color_system(file))
mylabel5 = Label(frame1,text="Set Angle :",)
e5=Entry(frame1,width=10,state='disable')
btn_rotate=Button(frame1,text='Rotate',state='disable',command=lambda :geometry(file))
mylabel6 = Label(frame1,text="X1 :",)
e6=Entry(frame1,width=10,state='disable')
mylabel7 = Label(frame1,text="Y1 :",)
e7=Entry(frame1,width=10,state='disable')
mylabel8 = Label(frame1,text="X2 :",)
e8=Entry(frame1,width=10,state='disable')
mylabel9 = Label(frame1,text="Y2 :",)
e9=Entry(frame1,width=10,state='disable')
btn_crop=Button(frame1,text='Crop',state='disable',command=lambda :decoper(file))
mylabel11 = Label(frame1,text="Set T  Valeur :")
e11=Entry(frame1,width=10,state='disable')
btn_merge=Button(frame1,text='Merge',state='disable',command=lambda :merge(file))


e1.grid(row=0,column=0,padx=10,pady=20,ipady=5,ipadx=6)
btn_get.grid(row=0,column=1,padx=10,ipadx=10,ipady=3,pady=20)
mylabel1.grid(sticky =W,row=1,column=0,padx=(20,0),columnspan=2,pady=20)
e2.grid(sticky =W,row=1,column=0,padx=(100,0),columnspan=2,pady=20,ipady=5,ipadx=6)
mylabel2.grid(sticky =W,row=1,column=0,padx=(220,0),columnspan=2,pady=20)
e3.grid(sticky =W,row=1,column=0,padx=(300,0),columnspan=2,pady=20,ipady=5,ipadx=6)
btn_resize.grid(sticky =E,row=1,column=0,padx=(0,30),ipadx=40,ipady=3,columnspan=2,pady=20)
btn_getcl.grid(sticky =W,row=2,column=0,padx=(20,0),pady=20,columnspan=2)
mylabel3.grid(sticky =W,row=2,column=0,padx=(180,0),columnspan=2,pady=20)
e4.grid(sticky =W,row=2,column=0,padx=(320,0),columnspan=2,pady=20,ipady=5,ipadx=30)
mylabel4.grid(sticky =W,row=3,column=0,padx=(20,0),columnspan=2,pady=20)
drop.grid(sticky =W,row=3,column=0,padx=(190,0),ipadx=18,ipady=3,pady=20)
btn_change.grid(sticky =W,row=3,column=0,padx=(360,0),ipadx=50,ipady=3,pady=20,columnspan=2,)
mylabel5.grid(sticky =W,row=4,column=0,padx=(20,0),columnspan=2,pady=20)
e5.grid(sticky =W,row=4,column=0,padx=(150,0),columnspan=2,pady=20,ipady=5,ipadx=6)
btn_rotate.grid(sticky =W,row=4,column=0,padx=(300,0),ipadx=40,ipady=3,columnspan=2,pady=20)
mylabel6.grid(sticky =W,row=5,column=0,padx=(20,0),columnspan=2,pady=20)
e6.grid(sticky =W,row=5,column=0,padx=(60,0),columnspan=2,pady=20,ipady=5,ipadx=6)
mylabel7.grid(sticky =W,row=5,column=0,padx=(160,0),columnspan=2,pady=20)
e7.grid(sticky =W,row=5,column=0,padx=(200,0),columnspan=2,pady=20,ipady=5,ipadx=6)
mylabel8.grid(sticky =W,row=5,column=0,padx=(300,0),columnspan=2,pady=20)
e8.grid(sticky =W,row=5,column=0,padx=(340,0),columnspan=2,pady=20,ipady=5,ipadx=6)
mylabel9.grid(sticky =W,row=5,column=0,padx=(440,0),columnspan=2,pady=20)
e9.grid(sticky =W,row=5,column=0,padx=(480,0),columnspan=2,pady=20,ipady=5,ipadx=6)
btn_crop.grid(sticky =W+E,row=6,column=0,padx=60,ipadx=40,ipady=3,columnspan=2,pady=(0,20))
mylabel11.grid(sticky =W,row=7,column=0,padx=(20,0),columnspan=2,pady=20)
e11.grid(sticky =W,row=7,column=0,padx=(150,0),columnspan=2,pady=20,ipady=5,ipadx=6)
btn_merge.grid(sticky =W,row=7,column=0,padx=(300,0),ipadx=40,ipady=3,columnspan=2,pady=20)

#-------------------------------------------------------------------------------------------------------

#------------------------Original----------------------------------------------------------------

mylabel12 = Label(frame2,text="Chose System Color :",)
blur = ['gaussian','s&p','poisson','salt','pepper']
click2 = StringVar(window)
click2.set("Type")
drop2 = OptionMenu(frame2, click2, *blur)
drop2.config(state='disable')
btn_bruit=Button(frame2,text='Start',state='disable',command=lambda :bruit(file))
mylabel13 = Label(frame2,text="Chose filtre :",)
filtre = ['Median','Moyenneur','Gaussian']
click3 = StringVar(window)
click3.set("Filtre")
drop3 = OptionMenu(frame2, click3, *filtre)
drop3.config(state='disable')
mylabel14 = Label(frame2,text="Chose dimenssion :",)
dimenssion = [3,5,7]
click4 = StringVar(window)
click4.set("Size")
drop4 = OptionMenu(frame2, click4, *dimenssion)
drop4.config(state='disable')
mylabel15 = Label(frame2,text="Set Sigma :")
e12=Entry(frame2,width=10,state='disable')
btn_filtre=Button(frame2,text='Filtre',state='disable',command=lambda :filtrage(file))
btn_dyn=Button(frame2,text='Normalise',state='disable',command=lambda :dynamic(file))
btn_ega=Button(frame2,text='Egalise',state='disable',command=lambda :egalisation(file))
btn_qual=Button(frame2,text='Get quality',state='disable',command=lambda :qualite(file))
mylabel16 = Label(frame2,text="MSE :")
e13=Entry(frame2,width=10,state='disable')
mylabel17 = Label(frame2,text="SNR :")
e14=Entry(frame2,width=10,state='disable')
mylabel18 = Label(frame2,text="PNSR :")
e15=Entry(frame2,width=10,state='disable')



mylabel12.grid(sticky =W,row=0,column=0,padx=(20,0),pady=20)
drop2.grid(sticky =W,row=0,column=0,padx=(190,0),ipadx=20,ipady=3,pady=20)
btn_bruit.grid(sticky =W,row=0,column=0,padx=(370,84),ipadx=50,ipady=3,pady=20)
mylabel13.grid(sticky =W,row=1,column=0,padx=(20,0),pady=20)
drop3.grid(sticky =W,row=1,column=0,padx=(140,0),ipadx=20,ipady=3,pady=20)
mylabel14.grid(sticky =W,row=1,column=0,padx=(300,0),pady=20)
drop4.grid(sticky =W,row=1,column=0,padx=(450,0),ipadx=20,ipady=3,pady=20)
mylabel15.grid(sticky =W,row=2,column=0,padx=(20,0),pady=20)
e12.grid(sticky =W,row=2,column=0,padx=(140,0),ipadx=20,ipady=3,pady=20)
btn_filtre.grid(sticky =W,row=2,column=0,padx=(300,0),ipadx=80,ipady=3,pady=20)
btn_dyn.grid(sticky =W,row=3,column=0,padx=(20,0),ipadx=80,ipady=3,pady=20)
btn_ega.grid(sticky =W,row=3,column=0,padx=(300,0),ipadx=80,ipady=3,pady=20)
#btn_qual.grid(sticky =W,row=4,column=0,padx=(20,0),ipadx=80,ipady=3,pady=20)

#-------------------------------------------------------------------------------------------------------

window.mainloop()